In [1]:
import numpy as np
from numpy.linalg import norm, inv
from sympy import ordered, Matrix
from sympy import hessian as h
from sympy.abc import x, y, z
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt

plt.rcParams['text.usetex'] = True
plt.rcParams['savefig.format'] = 'jpg'

In [2]:
def normalize(v):
    return v / norm(v, ord=2)


def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False


def quad_model(f_ex, grad_ex, hess_ex):
    """
    This function returns a lambda function that is a
    quadratic approximation of f at a point x.
    :param f_ex:
    f evaluated at x.
    :param grad_ex:
    The gradient of f at x.
    :param hess_ex:
    The hessian of f at x.
    :return:
    """
    return lambda pea: f_ex + pea@grad_ex + .5*pea@hess_ex@pea


def trust_region(x_init, func, m_func, grad_func, hess_func,
                 p_func, max_trust=5., tolerance=1e-8,
                 init_trust=None):
    """
    This function uses the dogleg method of the trust
    region to find the minimizer of a function of 2
    variables.
    :param x_init:
    A list of initial starting points for the optimization.
    :param func:
    The function we would like to optimize.
    :param m_func:
    The function we use to construct a quadratic
    model.
    :param grad_func:
    A function to compute the gradient of func.
    :param hess_func:
    A function to compute the hessian of func.
    :param p_func:
    The function we use to find a search direction
    that minimizes the quadratic model.
    :param max_trust:
    The maximum radius of the trust region.
    :param tolerance:
    :return:
    """
    x_solutions = [None]*x_init.shape[0]
    for n, x_0 in enumerate(x_init):
        x_sols = np.zeros(shape=(1_000, x_init.shape[1]))
        ex = np.copy(x_0).astype(dtype=np.float64)
        k = 0
        eta = np.random.uniform(0, .25)
        trust_radius = max_trust*.5 if init_trust is None else init_trust
        grad_x = grad_func(*ex)
        hess_x = hess_func(*ex)
        quad_x = m_func(func(*ex), grad_x, hess_x)

        while norm(grad_x, ord=2) > tolerance:
            x_sols[k] = ex
            p_approx = p_func(grad_x, hess_x, trust_radius)
            rho_k = (func(*ex) - func(*(ex + p_approx)))/\
                    (func(*ex) - quad_x(p_approx))
            if rho_k < .5:
                trust_radius *= .25
            elif (rho_k >= .5 and
                  np.abs(norm(p_approx) - trust_radius) < tolerance):
                trust_radius = min(2*trust_radius, max_trust)
            if rho_k > eta:
                ex += p_approx
                grad_x = grad_func(*ex)
                hess_x = hess_func(*ex)
                quad_x = m_func(func(*ex), grad_x, hess_x)
                k += 1
        x_solutions[n] = x_sols[:k]

    return x_solutions

def tau_min(p_u, p_fs, trust_radius):
    return lambda tau: norm(p_u + (tau - 1)*(p_fs-p_u)) - trust_radius


def bisection(a_low, a_high, func, tolerance=1e-8):
    while np.abs(a_high - a_low) > tolerance:
        m = .5 * (a_low + a_high)
        if np.sign(func(m)) != np.sign(func(a_high)):
            a_low = m
        else:
            a_high = m
    return .5 * (a_low + a_high)


def tau_comp(grad, hess, trust_radius):
    posi_def = grad.dot(hess).dot(grad)
    if posi_def <= 0:
        return 1
    else:
        uncon_min = (norm(grad, ord=2)**3)/(trust_radius*posi_def)
        return min(1, uncon_min)


def cauchy_point(grad, hess, trust_radius, tau_func=tau_comp):
    tau_k = tau_func(grad, hess, trust_radius)
    print(f"tau_k: {tau_k}")
    if is_pos_def(hess):
        return dogleg_step(grad, hess, trust_radius, tau_k)
    else:
        p_k = -trust_radius*normalize(grad)
        return tau_k*p_k


def dogleg_step(grad_x, hess_x, trust_radius):
    p_u = -(grad_x.dot(grad_x))/(grad_x.dot(hess_x).dot(grad_x))*grad_x
    p_fs = -inv(hess_x).dot(grad_x)
    if norm(p_u, ord=2) >= trust_radius:
        return trust_radius*normalize(p_u)
    elif norm(p_fs, ord=2) <= trust_radius:
        return p_fs
    else:
        m = tau_min(p_u, p_fs, trust_radius)
        tau = bisection(1,2, m)
        return p_u + (tau - 1)*(p_fs - p_u)


def make_grad_hess(funkshun, *args):
    g = funkshun(*args)

    vars = list(ordered(g.free_symbols))
    gradient = lambda func, frees: Matrix([func]).jacobian(frees)

    gradi = lambdify(vars, gradient(g, vars))
    hessi = lambdify(vars, h(g, vars))

    return lambda ex, why: gradi(ex, why).flatten(), hessi

In [3]:
f = lambda x1, x2: 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2
gradie, hessia = make_grad_hess(f, x, y)

In [10]:

x_init = np.zeros(shape=(6,2)).astype(dtype=np.float64)
x_init[0] = [-1.2, 1.]
x_init[1] = [2.8, 4.]
x_init[2] = [1.2, 1.2]
x_init[3] = [64., 89.]
x_init[4] = [-90., -100.]
x_init[5] = [200., -250.]

ks = np.ones((x_init.shape[0], 6))
for j, i in enumerate(range(50, 301, 50)):
    equis = trust_region(x_init, f, quad_model,
                         gradie, hessia,
                         dogleg_step, max_trust=i,
                         init_trust=1.)
    print(f"Max trust radius: {i}")
    for k, sol in enumerate(equis):
        print(f"Starting point: {sol[0]},\n"
              f"Minimum: {sol[-1]},\n"
              f"Total Iterations: {sol.shape[0]}\n"
              f"First 5:\n{sol[:5]}\n"
              f"Last 4:\n{sol[-4:]}")
        ks[k,j] = sol.shape[0]

print(f"\n{ks}\n")

Max trust radius: 50
Starting point: [-1.2  1. ],
Minimum: [1.         0.99999999],
Total Iterations: 24
First 5:
[[-1.2         1.        ]
 [-1.1752809   1.38067416]
 [-1.07407754  1.15207435]
 [-0.96464932  0.92729565]
 [-0.73010519  0.48572001]]
Last 4:
[[0.99109917 0.98164169]
 [0.99899574 0.99793014]
 [0.99998763 0.99997428]
 [1.         0.99999999]]
Starting point: [2.8 4. ],
Minimum: [1.00000001 1.00000001],
Total Iterations: 19
First 5:
[[2.8        4.        ]
 [2.37618752 4.90574996]
 [2.36695777 5.60240391]
 [2.26306222 5.11331732]
 [2.15304408 4.62557146]]
Last 4:
[[1.02252306 1.04536819]
 [1.00080454 1.00113802]
 [1.00006936 1.00013818]
 [1.00000001 1.00000001]]
Starting point: [1.2 1.2],
Minimum: [1.00000023 1.00000047],
Total Iterations: 8
First 5:
[[1.2        1.2       ]
 [1.19591837 1.43020408]
 [1.09933239 1.19961538]
 [1.06364326 1.13006327]
 [1.01292111 1.02343643]]
Last 4:
[[1.01292111 1.02343643]
 [1.00438977 1.00872604]
 [1.00006298 1.00010725]
 [1.00000023 1.0

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(np.log10(f(*equis[0].T)), '-k', label=r"$\vec{x}_0 = "
                                              f"({equis[0][0,0]}, "
                                              f"{equis[0][0,1]})$")
ax.set_xlabel(r"$k$", size=25)
ax.set_ylabel(r"$\log_{10}\left(f(\vec{x}_k)\right)$", size=25)
ax.tick_params(direction='in', width=2, length=8, labelsize=25)
ax.legend(loc='upper right', fontsize=25)
ax.set_title("Steepest Descent Algorithm", size=25)
ax.grid()

plt.show()

In [ ]:
for k, q in enumerate(equis[:2]):
    fig, ax = plt.subplots(figsize=(12, 8))

    ax.plot(np.log10(f(*q.T)), '-k',
            label=fr"$\vec{{x}}_0 = ({q[0,0]}, {q[0,1]})$")

    ax.grid()
    ax.set_xlabel(r"$k$", size=25)
    ax.set_ylabel(r"$\log_{10}\left(f(\vec{x}_k)\right)$", size=25)
    ax.tick_params(direction='in', width=2, length=8, labelsize=25)
    ax.set_title("Dogleg method, $\widehat{\Delta} = 300$", size=25)
    ax.legend(loc='upper right', fontsize=25);
    ax.set_xticks([i for i in range(0, q.shape[0], 5)])

    fig.savefig(f"HW3;Objective{k}")

In [12]:
minimum = [1., 1.]
for q in equis:
    w = q - minimum
    w = np.linalg.norm(w, ord=2, axis=1)
    coeff = np.polyfit(np.log(w[:-1]), np.log(w[1:]), 1)
    print(f"Rate of convergence; x0 = {q[0]}: {coeff[0]}")

Rate of convergence; x0 = [-1.2  1. ]: 1.7096584548064946
Rate of convergence; x0 = [2.8 4. ]: 1.667303158458797
Rate of convergence; x0 = [1.2 1.2]: 1.6819089199401087
Rate of convergence; x0 = [64. 89.]: 1.3357445881203478
Rate of convergence; x0 = [ -90. -100.]: 1.2881806845938344
Rate of convergence; x0 = [ 200. -250.]: 1.2304148974286027


In [ ]:
q = equis[0]

X = np.linspace(-2, 2, 5 * 101)
Y = np.linspace(-.1, 2, 5 * 101)
X, Y = np.meshgrid(X, Y)

Z = f(X, Y)
Z[Z > 400] = 400
fig, ax = plt.subplots(1, 1, figsize=(18, 14))

ax = plt.axes(projection='3d')
ax.plot(*q.T, f(*q.T), '-r', lw=4,
        label=f"Optimization Path\nDogleg Method\nSteps: {q.shape[0]}")
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='viridis', alpha=.5)

ax.text(*q[0], f(*q[0]) + 1,
        f"Starting point: $({q[0, 0]}, {q[0, 1]}, {f(*q[0]):3.2f})$",
        size=25, ha="right", va="bottom", color="red")
ax.text(*q[-1], f(*q[-1]),
        f"Minimum: $({q[-1, 0]:0.1f}, {q[-1, 1]:0.1f}, {f(*q[-1]):0.1f})$",
        size=25, ha="center", va="bottom", color="red")

ax.set_xlabel("$x$", size=30, labelpad=20)
ax.set_xticks([i - 2.0 for i in range(5)])
ax.set_ylabel("$y$", size=30, labelpad=20)
ax.set_yticks([.5*i for i in range(5)])
ax.set_zlabel("$f(x,y)$", size=30, labelpad=25)
ax.set_zticks([i*100 for i in range(5)])
ax.zaxis.set_rotate_label(False)

ax.tick_params(direction='in', width=2, length=8, labelsize=25)
ax.view_init(45, 230+60)
ax.legend(fontsize=20, loc='lower left')

ax.set_title("Rosenbrock's Function on "
            fr"$[{X.min()},{X.max()}]\times [{Y.min()},{Y.max()}]$", size=30);

fig.savefig(f"HW3;HardStart")

In [ ]:
q = equis[1]

X = np.linspace(0, 3, 5 * 101)
Y = np.linspace(0, 6, 5 * 101)
X, Y = np.meshgrid(X, Y)

Z = f(X, Y)
Z[Z > 3_000] = 3_000
fig, ax = plt.subplots(1, 1, figsize=(18, 14))

ax = plt.axes(projection='3d')
ax.plot(*q.T, f(*q.T), '-r', lw=4,
        label=f"Optimization Path\nDogleg Method\nSteps: {q.shape[0]}")
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='viridis', alpha=.5)

ax.text(*q[0], f(*q[0]) + 1,
        f"Starting point: $({q[0, 0]}, {q[0, 1]}, {f(*q[0]):3.2f})$",
        size=25, ha="center", va="bottom", color="red")
ax.text(*q[-1], f(*q[-1]),
        f"Minimum: $({q[-1, 0]:0.1f}, {q[-1, 1]:0.1f}, {f(*q[-1]):0.1f})$",
        size=25, ha="right", va="top", color="red")

ax.set_xlabel("$x$", size=30, labelpad=20)
ax.set_ylabel("$y$", size=30, labelpad=20)
ax.set_zlabel("$f(x,y)$", size=30, labelpad=25)
ax.zaxis.set_rotate_label(False)
ax.set_zticks([i*1000 for i in range(4)])

ax.tick_params(direction='in', width=2, length=8, labelsize=25)
ax.view_init(45, 230)
ax.legend(fontsize=20, loc='lower left')

ax.set_title("Rosenbrock's Function on "
            fr"$[{X.min()},{X.max()}]\times [{Y.min()},{Y.max()}]$", size=30);

fig.savefig(f"HW3;EasyStart")

In [ ]:
q = equis[1]

X = np.linspace(-2, 3, 5 * 101)
Y = np.linspace(-.1, 6, 5 * 101)
X, Y = np.meshgrid(X, Y)

Z = f(X, Y)
Z[Z > 2_000] = 2_000
fig, ax = plt.subplots(1, 1, figsize=(18, 14))
colors = ['red', 'blue']
ax = plt.axes(projection='3d')
for queue, col in zip(equis[:2], colors):
    ax.plot(*queue.T, f(*queue.T), ls='-', color=col, lw=4,
            label=f"Optimization Path\nDogleg Method\nSteps: {queue.shape[0]}")
    ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='viridis', alpha=.5)

    ax.text(*queue[0], f(*queue[0]) + 1,
            f"Starting point: $({queue[0, 0]}, {queue[0, 1]}, {f(*queue[0]):3.2f})$",
            size=25, ha="center", va="bottom", color=col)
    ax.text(*queue[-1], f(*queue[-1]),
            f"Minimum: $({queue[-1, 0]:0.1f}, {queue[-1, 1]:0.1f}, {f(*queue[-1]):0.1f})$",
            size=25, ha="right", va="top", color=col)

ax.set_xlabel("$x$", size=30, labelpad=20)
ax.set_ylabel("$y$", size=30, labelpad=20)
ax.set_zlabel("$f(x,y)$", size=30, labelpad=25)
ax.zaxis.set_rotate_label(False)
ax.set_zticks([i*1000 for i in range(3)])

ax.tick_params(direction='in', width=2, length=8, labelsize=25)
ax.view_init(45, 230)
ax.legend(fontsize=20, loc='lower left')

ax.set_title("Rosenbrock's Function on "
            fr"$[{X.min()},{X.max()}]\times [{Y.min()},{Y.max()}]$", size=30);
